# Timing Cost Computation

This notebook goes through each bloq example and calls `report_on_cost_timings`, which currently times how long it takes to do the `QubitCount` cost key. This uses the `ExecuteWithTimeout` fixture.

In [ ]:
from qualtran_dev_tools.execute_with_timeout import ExecuteWithTimeout
from qualtran_dev_tools.bloq_report_card import report_on_cost_timings
from qualtran_dev_tools.bloq_finder import get_bloq_examples

In [ ]:
bes = get_bloq_examples()

# Imports to exclude certain bloqs, see following comment
from qualtran.bloqs.multiplexers.apply_gate_to_lth_target import ApplyGateToLthQubit

In [ ]:
exec = ExecuteWithTimeout(timeout=20., max_workers=4)
for i, be in enumerate(bes):

    if be.bloq_cls == ApplyGateToLthQubit:
        # This bloq uses a lambda function as one of its attributes, which
        # can't be pickled and used with multiprocessing.
        continue
    
    exec.submit(report_on_cost_timings, kwargs=dict(name=be.name, cls_name=be.bloq_cls.__name__, bloq=be.make()))

records = []
while exec.work_to_be_done:
    kwargs, record = exec.next_result()
    print('\r', f'{exec.work_to_be_done:5d} remaining', end='', flush=True)
    
    if record is None:
        records.append({
            'name': kwargs['name'],
            'cls': kwargs['cls_name'],
            'err': 'Timeout',
        })
    else:
        records.append(record)

import pandas as pd
df = pd.DataFrame(records)

## Slowest

This prints the total number of bloq examples considered and then summarizes the 5 slowest-to-compute bloq examples.

In [ ]:
print(len(df))
df.sort_values(by='qubitcount_dur', ascending=False).head()

## Errors and timeouts

These bloq examples either time-out or encounter errors in the qubit computation.

In [ ]:
df[df['qubitcount_dur'].isna()]

In [ ]:
for i, row in df[df['qubitcount_dur'].isna()].iterrows():
    print("### `{}`".format(row['name']))
    print("{}\n".format(row["err"]))

## Timeouts

These examples specifically time out. 

In [ ]:
df[df['err'] == 'Timeout']

## Investigation

Individual bloq examples can be investigated. Strangely, hubbard_time_evolution_by_gqsp times out when run through the fixture but appears reasonably quick when run directly.

In [ ]:
def get_bloq_example(name):
    results = [be for be in bes if be.name == name]
    
    if len(results) == 1:
        return results[0]
    if len(results) > 1:
        raise ValueError("Found more than one result for the query")
    if len(results) == 0:
        raise KeyError(f"The bloq example {name} was not found")    

In [ ]:
from qualtran.drawing import show_call_graph
be = get_bloq_example('hubbard_time_evolution_by_gqsp')
bloq = be.make()
show_call_graph(bloq, max_depth=1)

In [ ]:
%%timeit
from qualtran.resource_counting import get_cost_value, QubitCount

get_cost_value(bloq, QubitCount())